In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.lstm_baseline_functions import (
    lstm_hyperparameter_search,
    obtain_path,
)

In [3]:
output_dir = "therapist_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-22 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-22 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-22 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-22 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-22 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# Baseline: LSTM classification

In [7]:
num_epochs = 100
hidden_dim_sizes = [100, 200, 300, 384]
num_layers = 1
bidirectional = True
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"

## `history_length=20`

In [8]:
size = 20
(
    bilstm_history_20_kfold,
    best_bilstm_history_20_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    output_dim=output_dim_therapist,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/lstm_history_20_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in therapist_talk_type_output/lstm_history_20_focal_2_kfold_best_model.csv


In [9]:
bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/tmp/ipykernel_1948104/3632055421.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  bilstm_history_20_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  precision  \
hidden_dim dropout_rate learning_rate                                  
100        0.1          0.0001         0.637318  0.597015   0.599630   
                        0.0005         0.688425  0.650007   0.650828   
                        0.0010         0.701139  0.665687   0.667653   
           0.5          0.0001         0.636939  0.597718   0.600033   
                        0.0005         0.684124  0.644626   0.647388   
                        0.0010         0.696711  0.659085   0.660946   
200        0.1          0.0001         0.644845  0.607110   0.607992   
                        0.0005         0.696015  0.657824   0.658825   
                        0.0010         0.694687  0.657890   0.659592   
           0.5          0.0001         0.644023  0.603433   0.605458   
                        0.0005         0.696458  0.657344   0.658687   
                        0.0010         0.699178  0.662338   0.664193   
300        0.1          0.0001         0.649146  0.609655   0.613280   
                        0.0005         0.696078  0.659120   0.660770   
                        0.0010         0.698102  0.659558   0.663088   
           0.5          0.0001         0.644845  0.604865   0.607585   
                        0.0005         0.692347  0.652740   0.656084   
                        0.0010         0.698039  0.659016   0.661033   
384        0.1          0.0001         0.658950  0.620304   0.621108   
                        0.0005         0.693675  0.654710   0.655742   
                        0.0010         0.698355  0.659487   0.662186   
           0.5          0.0001         0.644972  0.607331   0.609884   
                        0.0005         0.693548  0.655443   0.657774   
                        0.0010         0.695003  0.656405   0.659890   

                                         recall  valid_accuracy  valid_f1  \
hidden_dim dropout_rate learning_rate                                       
100        0.1          0.0001         0.596691        0.682911  0.651049   
                        0.0005         0.650461        0.724473  0.693701   
                        0.0010         0.666581        0.732331  0.704217   
           0.5          0.0001         0.598455        0.687922  0.657178   
                        0.0005         0.643834        0.725158  0.694015   
                        0.0010         0.659065        0.733175  0.703367   
200        0.1          0.0001         0.608809        0.687025  0.654888   
                        0.0005         0.658162        0.728903  0.698673   
                        0.0010         0.658517        0.733650  0.704730   
           0.5          0.0001         0.603947        0.684705  0.651817   
                        0.0005         0.656986        0.727532  0.696161   
                        0.0010         0.662200        0.733122  0.703364   
300        0.1          0.0001         0.608639        0.693882  0.662074   
                        0.0005         0.659050        0.729378  0.698251   
                        0.0010         0.658252        0.731382  0.699679   
           0.5          0.0001         0.604659        0.690823  0.659145   
                        0.0005         0.651792        0.728534  0.697096   
                        0.0010         0.658549        0.731065  0.700652   
384        0.1          0.0001         0.620924        0.695464  0.662189   
                        0.0005         0.655269        0.728217  0.697178   
                        0.0010         0.658211        0.735812  0.705756   
           0.5          0.0001         0.609272        0.688186  0.656183   
                        0.0005         0.654771        0.727057  0.695405   
                        0.0010         0.655687        0.731118  0.700776   

                                       valid_precision  valid_recall     k  \
hidden_dim dropout_rate learning_rate                                        
100    

In [10]:
best_bilstm_history_20_kfold

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,bidirectional,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.696395,0.657426,"[0.7137521222410866, 0.44778362133734034, 0.62...",0.659346,"[0.725828729281768, 0.4508320726172466, 0.5977...",0.656555,"[0.7020708082832331, 0.44477611940298506, 0.65...",None,0.733070,...,True,384,0.1,0.001,1,focal,2,True,5,64
0,None,0.702657,0.662972,"[0.7219578518014955, 0.45351473922902497, 0.62...",0.664699,"[0.7349480968858132, 0.45941807044410415, 0.60...",0.662002,"[0.7094188376753507, 0.44776119402985076, 0.65...",None,0.739241,...,True,384,0.1,0.001,12,focal,2,True,5,64
0,None,0.696015,0.658061,"[0.704483695652174, 0.45349730976172176, 0.626...",0.662512,"[0.7166551485832757, 0.4675118858954041, 0.585...",0.656077,"[0.6927187708750835, 0.44029850746268656, 0.67...",None,0.735127,...,True,384,0.1,0.001,123,focal,2,True,5,64


In [11]:
best_bilstm_history_20_kfold["f1"].mean()

0.6594865308186346

In [12]:
best_bilstm_history_20_kfold["precision"].mean()

0.6621856676013539

In [13]:
best_bilstm_history_20_kfold["recall"].mean()

0.6582113701318494

In [14]:
np.stack(best_bilstm_history_20_kfold["f1_scores"]).mean(axis=0)

array([0.71339789, 0.45159856, 0.62710085, 0.84584883])

In [15]:
np.stack(best_bilstm_history_20_kfold["precision_scores"]).mean(axis=0)

array([0.72581066, 0.45925401, 0.59578063, 0.86789737])

In [16]:
np.stack(best_bilstm_history_20_kfold["recall_scores"]).mean(axis=0)

array([0.70140281, 0.44427861, 0.66212989, 0.82503417])

## `history_length=50`

In [ ]:
size = 50
(
    bilstm_history_50_kfold,
    best_bilstm_history_50_kfold,
    _,
    __,
) = lstm_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="main_therapist_behaviour",
    embeddings=sbert_embeddings,
    y_data=y_data_therapist,
    output_dim=output_dim_therapist,
    history_lengths=[size],
    hidden_dim_sizes=hidden_dim_sizes,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=therapist_index,
    split_ids=therapist_transcript_id,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/lstm_history_{size}_focal_{gamma}_kfold.csv",
    verbose=False,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
bilstm_history_50_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

In [ ]:
best_bilstm_history_50_kfold

In [ ]:
best_bilstm_history_50_kfold["f1"].mean()

In [ ]:
best_bilstm_history_50_kfold["precision"].mean()

In [ ]:
best_bilstm_history_50_kfold["recall"].mean()

In [ ]:
np.stack(best_bilstm_history_50_kfold["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_50_kfold["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_bilstm_history_50_kfold["recall_scores"]).mean(axis=0)